In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()

In [ ]:
import pandas as pd
newsgroups_df = pd.DataFrame(newsgroups.data, columns=["text"])
newsgroups_df["newsgroup"] = [newsgroups.target_names[t] for t in newsgroups.target]
newsgroups_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cvectors = cv.fit_transform(newsgroups_df["text"])

In [ ]:
cvectors

In [ ]:
cvectors.data.nbytes

In [ ]:
freq = cvectors.sum(axis=0).A[0]
freq

In [ ]:
top10 = freq.argsort()[::-1][0:10]
top10

In [ ]:
voc = cv.get_feature_names_out()
{ voc[pos]: freq[pos] for pos in top10 }

In [ ]:
cv = CountVectorizer(min_df=5)
cvectors = cv.fit_transform(newsgroups_df["text"])
cvectors

In [ ]:
cvn = CountVectorizer(ngram_range=(1,2), min_df=5)
cvectorsn = cvn.fit_transform(newsgroups_df["text"])
cvectorsn

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tvectors = tfidf.fit_transform(cvectors)
tvectors